<a href="https://colab.research.google.com/github/TaenorFloat/Projet-Annuel/blob/master/G%C3%A9n%C3%A9rateurDeBiographies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
! pip install sparqlwrapper
! python -m spacy download fr
! python -m spacy download fr_core_news_sm

     |████████████████████████████████| 348kB 14.4MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 13.1MB 757kB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.1.0-cp36-none-any.whl size=13156209 sha256=503826ca3b0ec8ca81306aeadaa9d6c47d767289e78d17504af60eafa031e9f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_b9jik5/wheels/ab/82/2a/61dd0ff02e22f10eef65a5aa35453a0eb745c84b4c874b612f
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [0]:
# Modules d'archivage et traitement de fichiers
import io
import csv
import json
import pandas

# Module de gestion des date sous python
import datetime

# Modules de requêtage HTTP et Webscraping
import requests
from bs4 import BeautifulSoup

# Modules d'accès KB's et SPARQL
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

# Modules de traitements linguistiques, et réseaux neuronals
import spacy
import fr_core_news_sm

# Modules de traitements linguistiques, et réseaux neuronals
import keras
import numpy as np
import random


# Projet Annuel : *Génération de Biograhies Automatique à l'aide de Réseaux de Neurones (LSTM)*

Afin de réaliser ce projet, nous nous aiderons de bases de connaissances (*KB's*) pour l'extraction de nos données et par la suite l'entrainement de nôtre RN.

Pour cela, nous cinderons en deux grandes étapes la réalisation de ce travail que sont :


1.   Extraction et épuration des données des KB's.
2.   Entrainement du RN.


# **Première Etape :**  Extraction des données

Après une étude de la conception des deux grandes bases de connaissances que sont *DBPédia* et *WikiData*, nous sommes arrivés à la conclusion qu'une utilisation exclusive de l'une des deux bases de connaissances ne serait pas optimale, et donc nous opterons par la suite pour une combinaison des deux, et cela comme suit :


1.   L'extraction des propriétés se fera de *WikiData*.
2.   L'extraction des présentations/abstract se fera de *DBPédia*.

Pour se faire, nous utiliseront le langage d'intérogation de bases de connaissances *SparQL*, et nous implémenterons se dernier via *Python*

In [0]:
"""
  Comme spécifié plus haut, la première étape de nôtre étude est la construction du dataset 'fr', et pour cela nous aurons besoins
    d'extraire des données de grandes bases de connaissances, combiné à cela, un peu de web scraping.
  
  En suivant une approche méthodologique, nous commencerons tout d'abord par la définition des différents algorithmes utiles à nos
    travaux, que sont les suivants:

    ○ Intérrogation et extraction des données des différentes KB's.
    ○ Création et sauvegarde des fichiers de dataset.
    ○ Formatage et combinaisons des différents fichiers de données.
"""

# https://rdflib.github.io/sparqlwrapper/

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [0]:
def create_abstracts_file():
  # Nous créons ensuite un nouveau dictionnaire, dont la charge est l'association de chaque entitées avec son abstract.
  abstracts = {}
  endpoint_url = 'http://fr.dbpedia.org/sparql'

  # Définition de la requête SPARQL à exécuter. (nous restreignons de lignes remontées dû à la latence serveur) 
  query = """
    PREFIX dbo:<http://dbpedia.org/ontology/>
    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>

    select ?acteurLabel ?abstract 
    where {
      ?acteur a dbo:Actor .
      ?acteur dbo:abstract ?abstract .
      ?acteur rdfs:label ?acteurLabel .
      FILTER (LANG(?abstract)='fr' and LANG(?acteurLabel)='en')
    } LIMIT 20000"""

  results = get_results(endpoint_url, query)

  # Ici nous déclarons/créons un fichier csv porterant le nom de "actors_abstracts" et contenant une association: nom => abstract.
  with open('drive/My Drive/Colab Notebooks/actors_abstracts.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Nous insérons dabord l'entête de nôtre fichier, pour une meilleure lisibilité.
    writer.writerow([
          'name',
          'abstract'
    ])
    
    for result in results["results"]["bindings"]:
      writer.writerow([
          result["acteurLabel"]["value"],
          result["abstract"]["value"]
      ])


In [0]:
# Cette fonction sert au formatage des dates.
def mounth_converter(mounth):
  if mounth == "01":
    return "janvier"
  elif mounth == "02":
    return "février"
  elif mounth == "03":
    return "mars"
  elif mounth == "04":
    return "avril"
  elif mounth == "05":
    return "mai"
  elif mounth == "06":
    return "juin"
  elif mounth == "07":
    return "juillet"
  elif mounth == "08":
    return "août"
  elif mounth == "09":
    return "septembre"
  elif mounth == "10":
    return "octobre"
  elif mounth == "11":
    return "novembre"
  else:
    return "décembre" 

# Pourquoi une utilisation combinée de deux bases de connaissances ?

Pour justifier un tel choix, il nous faut tout d'abord introduire les structures de données que proposent *Wikidata* et *DBPédia* :

# **WikiData**
Cette KB est très populaire du fait de sa liaison directe avec l'encyclopédie universelle et collaborative qu'est Wikipédia, ainsi que sa facilité d'utilisation. WikiData propose 78_979_737 entités, contrairement à Wikipédia qui elle n'en propose que 30 millions d'articles.
Malgré ces avantages, WikiData ne propose aucun résumé de ses entités, s'appuyant principalement sur le référencement de ses articles sur Wikipédia, mais comme vous l'aurez sans doute remarqué, pas toute les entités définies sur WikiData sont présentes sur l'encyclopédie Wikipédia, et cela nous amène à nôtre prochain point.

# **DBPédia**
Cette dernière fut conçue par ses auteurs comme le « noyaux du Web émergent de l'open data ». En 2010, elle décrivait plus 3,4 millions d'entités distinctes. De nos jours, cette dernière compte plus de 3 milliards d'informations (RDF).
DBPédia, s'étant principalement basée sur Wikipédia lors de sa construction, celle-ci contient entre autre de petits paragraphes dans différentes langues décrivants l'entité concernée. Mais dû au fait, d'un certains niveau de difficulté rencontrée lors de son exploitation, nous ne pûmes nous reposer exclusivement sur cette dernière.

# **Idée derrière la combinaison**
Comme précisée plus haut, *DBPédia* se basant plus sur une construction de graphe ontologique qu'un simple filtrage de données, il nous fut presque impossible d'en tirer le maximum. Pour cela, nous avons dû nous adapter à la situation, et procéder de la sorte :


1.   Extraction de 20_000 instances de { "nom" : "abstract" } à partir de *DBPédia*.
2.   Boucler sur ces noms, afin d'accéder à la page *Wikipédia*, si cette dernière existe, pour enfin pouvoir utiliser du "web scraping" dans le but d'extraire l'id *WikiData* associé à l'entité.
3.   Intéroger *WikiData* pour l'extraction des propriétés qui nous intéressent.



In [0]:
def create_dataset():
  # List contenant les noms d'acteurs retenus.
  data_name = {}

  # 1: Extraction des noms du fichier csv et les mettre dans une liste.
  with open('drive/My Drive/Colab Notebooks/Projet Annuel/dataset/actors_abstracts.csv', 'r', encoding="utf-8") as actor_abs:
    csvReader = csv.DictReader(actor_abs)
    for row in csvReader:
      data_name[ row['name'] ] = row['abstract']

  print("Noms enregistrés avec succès !!!")

  # List contenant les ID's de nos acteurs.
  identifiers = []

  tmp = 1

  lignes = []

  # 2: Boucler sur les noms, et extraire la page wikipédia.
  for name in data_name:

    # Tout URL Wikipédia possède le format suivant :
    r = requests.get("https://fr.wikipedia.org/wiki/"+name)

    # Extraction de l'article sous son format HTML.
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      # Le lien WikiData étant dans une balise <li> du site, nous allons extraire cette dernière.
      balise = soup.find('li', attrs={ 'id':'t-wikibase' })
      # L'identifiant WikiData d'une entité est toujours inscrit en fin de lien, à partir de l'emplacement 49.
      identifiers.append(balise.find('a')['href'][49:])

      print("Acteur n° : ",tmp)
      tmp += 1
      
    except AttributeError:
      # Si l'on ne trouve pas un lien directement, nous supprimant le tuple concerné.
      lignes.append(name)
  
  for nom in lignes:
    data_name.pop(nom)

  print("Identifiants enregistrés avec succès !!!")

  # Dictionnaire contenant les noms, dates et lieux de naissance.
  dico = {}

  # Intérroger WikiData pour extraire les informations scalaires de nos acteurs.
  for id in identifiers:
    # Définition de la requête SPARQL.
    query = """SELECT ?acteurLabel ?birthdateLabel ?birthplaceLabel
      WHERE {
        VALUES ?acteur {
          wd:"""+id+"""
        }
        
        ?acteur wdt:P569 ?birthdateLabel .
        ?acteur wdt:P19 ?birthplace .
        
        SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
      }"""

    endpoint_url = "https://query.wikidata.org/sparql"

    result = get_results(endpoint_url, query)

    print("Requête exécutée avec succès !!!")

    # List contenant tous les postes occupés par un acteur.
    occup = []
    
    query = """# acteur
      SELECT ?acteurLabel ?occupationLabel WHERE {
        VALUES ?acteur {
          wd:"""+id+"""
        }
        ?acteur wdt:P106 ?occupation .
        SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
      }"""

    result_ = get_results(endpoint_url, query)

    print("Professions enregistrées avec succès !!!")

    for _ in result_["results"]["bindings"]:
      # En utilisant un tableau associatif, nous réglons aussi le problème des doublons.
      occup.append(_['occupationLabel']['value'])

    endpoint_url = "http://dbpedia.org/snorql/"

    query = """"
      PREFIX dbo:<http://dbpedia.org/ontology/>
        SELECT ?titre WHERE {
        ?acteur foaf:name """+result["results"]["bindings"][0]["acteurLabel"]["value"]+"""@en.
        ?film dbo:starring ?acteur.
        ?film foaf:name ?titre.
        filter(LANG(?titre)="en")
      }"""

    m_result = get_results(endpoint_url, query)

    movies = []
    
    for _ in result["results"]["bindings"]:
      # On utilisant un tableau associatif, nous réglons aussi le problème des doublons.
      movies.append(_['titre']['value'])

    print("Films enregistrés avec succès !!!")

    try:

      date = str(datetime.datetime.strptime(result["results"]["bindings"][0]["birthdateLabel"]["value"], "%Y-%m-%dT%H:%M:%SZ"))[:10].split('-')
      date[1] = mounth_converter(date[1])

      date = date[2]+' '+date[1]+' '+date[0]

      dico[id] = [
              result["results"]["bindings"][0]["acteurLabel"]["value"],
              date,
              result["results"]["bindings"][0]["birthplaceLabel"]["value"],
              occup,
              movies,
              data_name[result["results"]["bindings"][0]["acteurLabel"]["value"]]
      ]

    except ValueError:
      # En cas d'inconformité de la donnée, cette dernière ne sera aps sauvegardée.
      continue

  print("Sauvegarde sans du dataset en cours...")

  # Enfin, nous combinons nos fichiers, pour construire un dataset complet sous format json.
  with open('drive/My Drive/Colab Notebooks/Projet Annuel/dataset/dataset.txt', 'w') as outfile:
    json.dump(occupations_by_id, outfile, ensure_ascii=False)

  print("Sauvegarde effectuée avec succès !!!")


In [0]:
%%time
create_dataset()

# **Deuxième Etape :** Epuration des données (*data cleaning*)
Après extraction des données, nous nous intéressons désormais au nettoyage de ces dernières, et cela via 2 procédés que sont :


1.   Correspondance des données et leurs labels dans les abstracts. (*matching*)
2.   Extraction d'un paterne de succession récurent entre les différentes propriétées. (*graph*)


In [0]:
def load_dataset():
  dataset_json = open('drive/My Drive/Colab Notebooks/Projet Annuel/dataset/dataset.txt', 'r', encoding='utf-8')
  dataset_dict = json.load(dataset_json)
  dataset_json.close()

  return dataset_dict

In [0]:
def corresponding_data():

  # Dictionnaire contenant nôtre dataset.
  dataset = load_dataset()

  # Pour chaque acteur.
  for _ in dataset:

    # Un attribut nous permettant de nous situer dans l'enregistrement.
    property = 0

    # On boucle sur ses propriétés.
    for propriete in dataset.get(_) :
      # Si ce sont des films ou des professions, on devra les parcourir aussi.
      if type(propriete) == list:
        for sous_propriete in propriete:
          if property == 3 : # On est aux professions.
            dataset[_][5] = dataset[_][5].replace(sous_propriete, "<PROPERTY_OCCUPATIONS>")
          elif property == 4 : # On est aux films.
            dataset[_][5] = dataset[_][5].replace(sous_propriete, "<PROPERTY_MOVIES>")
      else:
        if property == 0 : # Nom.
          dataset[_][5] = dataset[_][5].replace(propriete, "<PROPERTY_NAME>")
        elif property == 1 : # Date de naissance.
          dataset[_][5] = dataset[_][5].replace(propriete, "<PROPERTY_BIRTHDATE>")
        elif property == 2 : # Lieu de naissance.
          dataset[_][5] = dataset[_][5].replace(propriete, "<PROPERTY_BIRTHPLACE>")
        
      # Pour se situer dans le dictionnaire.
      property += 1

  return dataset

In [0]:
dataset = corresponding_data()

# **Troisième Etape :** Conception du modèle des réseaux de neurones & Entrainement

Dans ce qui va suivre, une partie conceptuelle ainsi qu'applicative "recyclée" dans nôtre projet, faute de temps, nous ne pûmes totalement la réétudier ni l'adapter au mieux à nôtre projet. Pour de plus amples informations veuillez vous référer à Belkacemi Ryad & Kesouri Manil.

L'algorithme qui suit consiste à lire nôs instances, et essayer de générer un semblent de texte conforme au format d'une biographie.

In [0]:

with open('drive/My Drive/Colab Notebooks/dataset/abstract_dataset.txt', encoding='utf-8') as f:
  text = f.read().lower()

chars = sorted (list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

sentences = []
next_chars = []

for i in range(0, len(text)-40, 3):
  sentences.append(text[i: i+40])
  next_chars.append(text[i+40])

x = np.zeros((len(sentences), 40, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  
  y[i, char_indices[next_chars[i]]] = 1


model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer= keras.optimizers.RMSprop(lr=0.01))


In [0]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  
  return np.argmax(probas)

In [0]:
model.fit(x, y, batch_size=18, epochs=60)

start_index = random.randint(0, len(text)-40-1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
  generated = ''
  sentence = text[start_index: start_index+40]
  generated += sentence

  for i in range(600):
    x_pred = np.zeros((1, 40, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

with open('drive/My Drive/Colab Notebooks/biographie.txt', 'w') as f:
  f.write(generated)